# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [10]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=5)
lngs = np.random.uniform(low=-180.000, high=180.000, size=5)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    #If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


5

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [13]:
# set up lists to hold response info
lat = []
temp_max = []
hum = []
cloud = []
wind = []
long = []
city_api = []
country = []
datetime_api = []

# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = f"{url}appid={api_key}&units={units}&q="

# Viewing the straight Json
# weather_response = requests.get(query_url + city)
# weather_json = weather_response.json()
# city_id = weather_json["id"]
# city_name = weather_json["name"]

    # Loop through the list of cities and perform a request for data on each
for city in cities:
        weather_response = requests.get(query_url + city)
        weather_json = weather_response.json()
        #Error handling
        try:
            city_id = weather_json["id"]
            city_name = weather_json["name"]
        
        # Add a one second interval between queries to stay within API query limits
            time.sleep(1)
    
            response = requests.get(query_url + city).json()
            lat.append(response['coord']['lat'])
            long.append(response['coord']['lon'])
            temp_max.append(response['main']['temp_max'])
            hum.append(response['main']['humidity'])
            cloud.append(response['clouds']['all'])
            wind.append(response['wind']['speed'])
            country.append(response['sys']['country'])
            city_api.append(response["name"])
            datetime_api.append(response["dt"])
            
            print(f"Retrieving information for City: {city_name} and City ID is: {city_id} ")
            
        except (KeyError, IndexError):
            print(f"Missing information for City: {city_name}. Skipping city.")
        
print(f"The latitude information received is: {lat}")
print(f"The latitude information received is: {long}")
print(f"The temperature information received is: {temp_max}")
print(f"The humidity information received is: {hum}")
print(f"The cloudyiness information received is: {cloud}")
print(f"The wind information received is: {wind}")
print(f"The city information received is: {city_api}")
#weather_json


Retrieving information for City: Belyy Yar and City ID is: 1510377 
Retrieving information for City: Shebunino and City ID is: 468902 
Retrieving information for City: Cidreira and City ID is: 3466165 
Retrieving information for City: Peshkovo and City ID is: 510539 
Retrieving information for City: Bluff and City ID is: 2175403 
The latitude information received is: [53.6, 57.66, -30.17, 47.02, -23.58]
The latitude information received is: [91.39, 40.04, -50.22, 39.39, 149.07]
The temperature information received is: [33.51, 48.27, 73.11, 53.31, 66.99]
The humidity information received is: [69, 84, 77, 69, 29]
The cloudyiness information received is: [0, 100, 0, 95, 0]
The wind information received is: [2.24, 13.46, 17.45, 6.68, 7.41]
The city information received is: ['Belyy Yar', 'Shebunino', 'Cidreira', 'Peshkovo', 'Bluff']


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [17]:
weather_dict = {
    "Longitude": long,
    "Latitude": lat,
    "City": city_api,
    "Country": country,
    "Temp Max": temp_max,
    "Humidity": hum,
    "Cloudiness (%)": cloud,
    "Wind Speed (MPH)": wind,
    "Time": datetime_api, 
    }
weather_data = pd.DataFrame(weather_dict)
weather_data.head()    

,Longitude,Latitude,City,Country,Temp Max,Humidity,Cloudiness (%),Wind Speed (MPH),Time
0,91.39,53.60,Belyy Yar,RU,33.51,69,0,2.24,1570916989
1,40.04,57.66,Shebunino,RU,48.27,84,100,13.46,1570916990
2,-50.22,-30.17,Cidreira,BR,73.11,77,0,17.45,1570916921
3,39.39,47.02,Peshkovo,RU,53.31,69,95,6.68,1570916923
4,149.07,-23.58,Bluff,AU,66.99,29,0,7.41,1570916924


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_data["Latitude"], weather_data["Temp Max"], marker="o")

# Incorporate the other graph properties
plt.title("City Latitude Vs. Max Temperature 10/11/19")
plt.ylabel("Max Temperature (Fahrenheit)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("CityLatitudeVsTemp.png")

# Show plot
plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_data["Latitude"], weather_data["Humidity"], marker="o")

# Incorporate the other graph properties
plt.title("City Latitude Vs. Humidity 10/11/19")
plt.ylabel("Humidity %")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("CityLatitudeVsHumidity.png")

# Show plot
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_data["Latitude"], weather_data["Cloudiness (%)"], marker="o")

# Incorporate the other graph properties
plt.title("City Latitude Vs. Cloudiness (%) 10/11/19")
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("CityLatitudeVsCloudiness.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build a scatter plot for each data type
plt.scatter(weather_data["Latitude"], weather_data["Wind Speed (MPH)"], marker="o")

# Incorporate the other graph properties
plt.title("City Latitude Vs. Wind Speed (MPH) 10/11/19")
plt.ylabel("Wind Speed (MPH)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure
plt.savefig("CityLatitudeVsWindSpeed.png")

# Show plot
plt.show()